In [19]:
from Crypto.PublicKey import RSA
import base64

In [3]:
key1 = RSA.import_key(open("key1.pem").read())
key2 = RSA.import_key(open("key2.pem").read())

In [4]:
key1

RsaKey(n=25080356853331150673712092961488349508728123694382279186941974911344272809718201683391687288116618021523872262260746884803456249468108932413753368793568123710905490623939699616018064364038794824072468125668702688048418916712950393799664781694224559810656290997284081084848717062228887604668548576609649709572412523306016494962925450783098637867249337121156908328927249731928363360657779226929980928871118145919627109584218577535657544952661333527174942990937484743860494188129607347202336812042045820577108243818426559386634634103676467773122325120858908782192357380855678371737765634819794619802582481594876770433687, e=65537)

In [7]:
key2

RsaKey(n=25080356853331150673712092961488349508728123694382279186941974911344272809718201683391687288116618021523872262260746884803456249468108932413753368793568123710905490623939699616018064364038794824072468125668702688048418916712950393799664781694224559810656290997284081084848717062228887604668548576609649709572412523306016494962925450783098637867249337121156908328927249731928363360657779226929980928871118145919627109584218577535657544952661333527174942990937484743860494188129607347202336812042045820577108243818426559386634634103676467773122325120858908782192357380855678371737765634819794619802582481594876770433687, e=343223)

In [10]:
key1.n == key2.n

True

In [12]:
msg1 = "RBVdQw7Pllwb42GDYyRa6ByVOfzRrZHmxBkUPD393zxOcrNRZgfub1mqcrAgX4PAsvAOWptJSHbrHctFm6rJLzhBi/rAsKGboWqPAWYIu49Rt7Sc/5+LE2dvy5zriAKclchv9d+uUJ4/kU/vcpg2qlfTnyor6naBsZQvRze0VCMkPvqWPuE6iL6YEAjZmLWmb+bqO+unTLF4YtM1MkKTtiOEy+Bbd4LxlXIO1KSFVOoGjyLW2pVIgKzotB1/9BwJMKJV14/+MUEiP40ehH0U2zr8BeueeXp6NIZwS/9svmvmVi06Np74EbL+aeB4meaXH22fJU0eyL2FppeyvbVaYQ=="
msg2 = "TSHSOfFBkK/sSE4vWxy00EAnZXrIsBI/Y6mGv466baOsST+qyYXHdPsI33Kr6ovucDjgDw/VvQtsAuGhthLbLVdldt9OWDhK5lbM6e0CuhKSoJntnvCz7GtZvjgPM7JDHQkAU7Pcyall9UEqL+W6ZCkiSQnK+j6QB7ynwCsW1wAmnCM68fY2HaBvd8RP2+rPgWv9grcEBkXf7ewA+sxSw7hahMaW0LYhsMYUggrcKqhofGgl+4UR5pdSiFg4YKUSgdSw1Ic/tug9vfHuLSiiuhrtP38yVzazqOZPXGxG4tQ6btc1helH0cLfw1SCdua1ejyan9l1GLXsAyGOKSFdKw=="

In [13]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

In [14]:
egcd(key1.e, key2.e)

(1, 133132, -25421)

In [56]:
# Bezout coefficients
133132*key1.e - 25421*key2.e

1

In [80]:
# modular inverse
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

    
# pads a string with a '0' at the beginning if length is odd
def pad_even(x):
    return ('', '0')[len(x) % 2] + x


# ciphertext to integer
def cipher_b2n(c):
    c64 = base64.b64decode(c)
    tmp = ""
    for char in c64: # char is int
        tmp += pad_even(str(hex(char))[2:]) # add padded hex representation of char to tmp
    return int(tmp, 16) # convert tmp: hex string -> dec int


# integer to plaintext
def cipher_n2b(m):
    hex_m = hex(m)[2:] # remove "0x"
    return bytes.fromhex(hex_m).decode('utf-8') # bytes to ascii text conversion

In [88]:
e1 = key1.e 
e2 = key2.e
(_, s1, s2) = egcd(e1, e2)
c1 = cipher_b2n(msg1)
c2 = cipher_b2n(msg2)
n = key1.n 
if s1 < 0:
    s1 = -s1
    c1 = modinv(c1, n)
elif s2 < 0:
    s2 = -s2
    c2 = modinv(c2, n)
m = (pow(c1,s1,n) * pow(c2,s2,n)) % n
print(f"msg int: {m}\nplaintext: {cipher_n2b(m)}")

msg int: 29754482138600667824873548828684976503132905299031678355938493565
plaintext: HTB{C0mmon_M0dUlu5S_1S_b4D}


$m = c_1^{s_1} \times c_2^{s_2} \mod n$